# **PREDICCIONES AGREGADAS**

In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
import gzip

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  torch.set_printoptions(precision=decimals)
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [2]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 2
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15
SAVE_RESULTS = True
LOW_DATA = True

In [3]:
if READ_LOCAL_DATA:
  PATH = "Resources/Agregado/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Agregado/"

In [4]:
documents = ['minuteY','hourY','dayY']

if LOW_DATA:
    PATH += "LowData/"

with open(PATH+"minuteX"+".pkl", 'rb') as file:
    dataX = np.array(pickle.load(file)[0])

with open(PATH+documents[COMPUTED_OPTION]+".pkl", 'rb') as file:
    dataY = np.array(pickle.load(file)[0])



print('\033[1m' + "\t SHAPES" + '\033[0m')
print(dataX.shape)
print(dataY.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/Agregado/LowData/minuteX.pkl'

In [ ]:
#We split a test set for testing
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=TEST_SIZE)
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)
VALIDATION_INDEX = int(len(X_train)*VALIDATION_SIZE)
print("Examples for validation: \n", VALIDATION_INDEX)

In [ ]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[1]
WINDOW_SIZE = 1440

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(x) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128, activation="relu")(x) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

In [ ]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [ ]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train[:-VALIDATION_INDEX],
            y_train[:-VALIDATION_INDEX],
            epochs=10,
            verbose=0,
            batch_size=128,
            validation_data=(X_train[-VALIDATION_INDEX:], y_train[-VALIDATION_INDEX:]))

In [ ]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)


In [ ]:
if SAVE_RESULTS:
    file_path = 'Resources/Resultados/Agregado/'
    if LOW_DATA:
        file_path += "LowData/"
    documents = ['minuteY','hourY','dayY']
    file = file_path+documents[COMPUTED_OPTION]+"-predictions"+".pkl.gz"
    pickle.dump(predictions, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-test"+".pkl.gz"
    pickle.dump(y_test, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-X"+".pkl.gz"
    pickle.dump(X_test, gzip.open(file, 'wb'))

In [ ]:
print("Resultados comparando directamente las salidas")
print(y_test.shape)
print(predictions.shape)
print('\033[1m' + "MSE: " + str(mean_squared_error(y_test,predictions)) + '\033[0m')
print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test,predictions)) + '\033[0m')
print("Resultados con suma diaria de mets")
if COMPUTED_OPTION < 2:
    y_test_suma = np.sum(y_test,axis=1)
    predictions_suma = np.sum(predictions,axis=1)
    print(y_test_suma.shape)
    print(predictions_suma.shape)
    print('\033[1m' + "MSE: " + str(mean_squared_error(y_test_suma,predictions_suma)) + '\033[0m')
    print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test_suma,predictions_suma)) + '\033[0m')

In [ ]:
if COMPUTED_OPTION < 2:
    plot_predictions_vs_real(predictions[0],y_test[0])
    plot_predictions_vs_real(predictions_suma[:100],y_test_suma[:100])
else:
    plot_predictions_vs_real(predictions_suma[:100],y_test_suma[:100])
